In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] ='/content'
!kaggle datasets download -d jillanisofttech/brain-stroke-dataset
!unzip \*.zip && rm *.zip

  0% 0.00/47.2k [00:00<?, ?B/s]
100% 47.2k/47.2k [00:00<00:00, 46.9MB/s]
Archive:  brain-stroke-dataset.zip
  inflating: brain_stroke.csv        


In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.spatial import distance

class FuzzyKNN:
    def __init__(self, k=5, m=2):
        self.k = k
        self.m = m

    def fit(self, X_train, y_train):
        self.X_train = np.array(X_train)
        self.y_train = np.array(y_train)
        self.labels, self.y_train_indices = np.unique(y_train, return_inverse=True)

    def calculate_membership(self, x, cluster_centers):
        distances = np.linalg.norm(x - cluster_centers, axis=1)
        membership_values = 1.0 / np.sum((distances / distances[:, np.newaxis]) ** (2 / (self.m - 1)), axis=1)
        return membership_values




In [12]:
def predict(self, X_test):
    y_pred = []
    for test_point in X_test:
        cluster_centers = np.zeros((len(self.labels), self.X_train.shape[1]))
        for index, label in enumerate(self.labels):
            label_X_train = self.X_train[self.y_train_indices == index]
            distances = [distance.euclidean(test_point, train_point) for train_point in label_X_train]
            sorted_indices = np.argsort(distances)
            k_nearest_indices = sorted_indices[:self.k]
            cluster_centers[index] = np.mean(label_X_train[k_nearest_indices], axis=0)

        membership_values = self.calculate_membership(test_point, cluster_centers)
        # Ensure that the weights and list have the same length
        if len(membership_values) != len(self.y_train_indices):
            membership_values = np.pad(membership_values, (0, len(self.y_train_indices) - len(membership_values)), 'constant')
        prediction = np.argmax(np.bincount(self.y_train_indices, weights=membership_values))
        y_pred.append(self.labels[prediction])
    return np.array(y_pred)

In [16]:
# Load the dataset
data = pd.read_csv('/content/brain_stroke.csv')

# Drop rows with missing values
data = data.dropna()

# Convert categorical variables to numeric
label_encoder = LabelEncoder()
for column in ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']:
    data[column] = label_encoder.fit_transform(data[column])

# Split the dataset into features and target variable
X = data.drop(columns=['stroke'])  # Features
y = data['stroke']  # Target variable

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Perform feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize and fit the Fuzzy k-nearest neighbors model
fknn = FuzzyKNN(k=5, m=2)
fknn.fit(X_train, y_train)

# Make predictions on the test data
y_pred = fknn.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


AttributeError: 'FuzzyKNN' object has no attribute 'predict'

In [15]:
# Install the required module
!pip install scikit-fuzzy

# Import the required module
from skfuzzy import fcmeans

# Change the model initialization to use fcmeans instead of FuzzyKNN
fknn = fcmeans(X_train, 2, 2, error=1e-5, maxiter=1000, init=None)

# Fit the model
fknn.fit(X_train, y_train)

# Make predictions on the test data
y_pred = fcmeans.predict(fknn, X_test)[0]

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894078 sha256=133424b85121790e11797f88c1e4b9897a075ebb27391a591e879777a864685a
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


ImportError: cannot import name 'fcmeans' from 'skfuzzy' (/usr/local/lib/python3.10/dist-packages/skfuzzy/__init__.py)